In [ ]:
#导入必要的包
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from sklearn import preprocessing
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier
import seaborn as sns
import math
import warnings
from sklearn.decomposition import PCA
from sklearn import metrics
warnings.filterwarnings("ignore")
import sys
import warnings
from tqdm import tqdm

In [ ]:
#数据存放路径
trn_path = 'data/data22195/hy_round2_trn.zip'

In [ ]:
#将所有数据连接起来
def get_data(path, get_type=True):
    store_df_list = []
    z = zipfile.ZipFile(path, "r")
    for file in z.namelist()[1:]:
        f=z.open(file)
        df = pd.read_csv(f)
        #df[df.columns[3]]=df[df.columns[3]].map(v_deal)
        df['秒数']=df['time'].map(lambda x:int(x[0:2])*2678400+
        (int(x[2])*10+int(x[3]))*86400+(int(x[5])*10+int(x[6]))*3600+
        (int(x[8])*10+int(x[9]))*60+(int(x[11])*10+int(x[12])))
        f.close()
        store_df_list.append(df)
    z.close()
    new_df = pd.concat(store_df_list,axis=0)
    if get_type:
        new_df.columns = ['ship','x','y','v','d','time','label','秒数']
    else:
        new_df.columns = ['ship','x','y','v','d','time','秒数']
    if get_type:
        label_dict = {'拖网':0, '刺网':1, '围网':2}
        new_df['label'] = new_df['label'].map(label_dict)
    return new_df

In [ ]:
#获取训练数据集
train_data = get_data(trn_path)

###### 由于异常经纬度的来源较集中，选择使用存在异常经纬度(0，0)区域的众数进行填补

In [ ]:
#数据处理部分
def area_x(x):
    if x == 0:
        y = 24.325
    else:
        y = x
    return y

def area_y(x):
    if x == 0:
        y = 118.117
    else:
        y = x
    return y

def v_deal(x):
    if x < 0.32:
        y = 0
    elif x >= 12.73:
        y = 12.73
    else:
        y = x
    return y

def d_deal(x):
    if x == 360:
        y = 0
    else:
        y = x
    return y

train_data['x'] = train_data['x'].map(area_x)
train_data['y'] = train_data['y'].map(area_y)
train_data['v'] = train_data['v'].map(v_deal)
train_data['d'] = train_data['d'].map(d_deal)

###### 以三点构造拐角特征，以轨迹中连续的三个位置点判别拐角的类型，（直线型，小角转弯型，大角转弯型）
###### 在特征工程中针对拐角进行了统计特征提取

In [ ]:
#计算拐角特征
guai_list=[]
for i in train_data['ship'].unique():
    guai_feature=[]
    df=train_data[train_data['ship']==i][['x','y']]
    if len(df)%2!=0:
        for j in range(0,len(df)//2*2,2):
            dist1=(df['x'].iloc[j]-df['x'].iloc[j+1])**2+(df['y'].iloc[j]-df['y'].iloc[j+1])**2
            dist2=(df['x'].iloc[j+1]-df['x'].iloc[j+2])**2+(df['y'].iloc[j+1]-df['y'].iloc[j+2])**2
            dist3=(df['x'].iloc[j]-df['x'].iloc[j+2])**2+(df['y'].iloc[j]-df['y'].iloc[j+2])**2
            guai_feature.append(dist1+dist2-dist3)
        guai_list.append(guai_feature)
    if len(df)%2==0:
        for j in range(0,len(df)-2,2):
            dist1=(df['x'].iloc[j]-df['x'].iloc[j+1])**2+(df['y'].iloc[j]-df['y'].iloc[j+1])**2
            dist2=(df['x'].iloc[j+1]-df['x'].iloc[j+2])**2+(df['y'].iloc[j+1]-df['y'].iloc[j+2])**2
            dist3=(df['x'].iloc[j]-df['x'].iloc[j+2])**2+(df['y'].iloc[j]-df['y'].iloc[j+2])**2
            guai_feature.append(dist1+dist2-dist3)
        guai_list.append(guai_feature)

## 特征工程
######   1）、使用PCA算法变换x和y（旋转变换），分别统计每个ship中变换后的x和y的最大值(x_max,y_max)、最小值(x_min,y_min)、平均值(x_mean,y_mean)、    
######                                          和众数(x_mode,y_mode)、以及x_mode与y_mode相乘的交互特征（刻画渔船频繁出现的位置特征）；    
######                                          同理，对x,y的旋转特征(x_pca,y_pca)进行了同样的提取特征的操作。  
######   2）、添加对渔船频繁点刻画的特征。根据经纬度的精度，分别对将原纬度和PCA变换后的纬度乘10的3次幂，分别再加上原经度，得到sign和sign_pca用于后面的特征。
######                         分别计算sign和sign_pca的众数sign_mode与sign_pca_mode（刻画频繁出现的位置）；  
######   3）、根据EDA发现刺网渔船作业的分散性较围网与拖网低，即刺网渔船会在局部集中的海域活动，为刻画这种行为，  
######   用每个ship的sign的nunique(即渔船经过的不同位置的数量)除以len(sign)(记录的总数量)，以该特征描述渔船运动的分散性，得到sign_nunique;    
######   根据sign_nunique特征的分布，再继续对sign_nunique进行分箱处理，小于0.2 置0、大于0.2小于0.63置1、其余的置3;      
######   4）、分别计算每个ship中速度大于0小于等于2、大于2小于等于6、大于6小于等于10的比例和平均值，三种区间代表三种运动状态，  
######   以每个区间中的平均速度代表该船在该运动状态下的速度描述（根据速度的分布图选择的对速度的分箱方式）。      
######   5）、为了描述频繁点处的其他信息，统计了sign_mode的count特征(作为频繁点处的先验信息)，以及不同的运动状态的速度的均值
######  （即统计相同的频繁点处的3种运动状态速度的平均值）。   
######   6）、计算每个ship中角度的平均值。  
######   7）、利用原数据'time'特征，提取其‘分钟’特征，计算其最大值。  
######   8）、时间差的平均值。  
######   9）、时间差的分位数。  
######   10）、时间差的中位数。  
######   11）、以速度大于4的连续运动片段作为工作状态，计算每个ship中的工作状态片段并计算每个片段的位移作为作业间隔，统计每个ship中的作业间隔的中位数特征  
######   12）、数据处理：skew较大的特征sign_pca_mode使用log函数将其正态化；与速度相关的特征保留其两位有效数字。  
######   13）、设计了拐角特征，并统计每艘船的三种拐角形式的数量，均值作为特征。  
######   14）、总体模型特征为42个  

In [ ]:
def main(train_data):
    
    # 输入x,y的PCA变换特征
    from sklearn.decomposition import PCA
    xy_martix = train_data[['x', 'y']].values
    pca = PCA(n_components=2).fit(xy_martix)
    result = pca.transform(xy_martix)
    train_data['x_pca'] = result[:, 0]
    train_data['y_pca'] = result[:, 1]
    
    #位置的标识特征
    train_data['sign']=train_data['x']*10**3+train_data['y']
    train_data['sign_pca'] = train_data['x_pca'] * 10 ** 3 + train_data['y_pca']
    
    # 提取月日时分秒
    def extract_dt(df):
        df['time'] = pd.to_datetime(df['time'], format='%m%d %H:%M:%S')
        df['month'] = df['time'].dt.month
        df['day'] = df['time'].dt.day
        df['hour'] = df['time'].dt.hour
        df['weekday'] = df['time'].dt.weekday
        #df = df.drop(['time'], axis=1)
        return df

    train_data = extract_dt(train_data)
                
    def try_feature(df, train):
        def group_feature(df, key, target, aggs):
            agg_dict = {}
            for ag in aggs:
                agg_dict[f'{target}_{ag}'] = ag
            print(agg_dict)
            tm = df.groupby(key)[target].agg(agg_dict).reset_index()
            return tm

        t = group_feature(df, 'ship', 'x', ['max','min', 'mean'])
        train = pd.merge(train, t, on='ship', how='left')
        t = group_feature(df, 'ship', 'y', ['max','min', 'mean'])
        train = pd.merge(train, t, on='ship', how='left')
        
        t = df.groupby('ship')['x'].agg({'x_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['y'].agg({'y_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        
        #针对PCA变换后的经纬度构造统计特征
        t = group_feature(df, 'ship', 'x_pca', ['max', 'min', 'mean'])
        train = pd.merge(train, t, on='ship', how='left')
        t = group_feature(df, 'ship', 'y_pca', ['max', 'min', 'mean'])
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['x_pca'].agg({'x_pca_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['y_pca'].agg({'y_pca_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')

        for i in ['x_mode']:
            for j in ['y_mode']:
                train[i + '*' + j] = train[i] * train[j]
        for i in ['x_pca_mode']:
            for j in ['y_pca_mode']:
                train[i + '*' + j] = train[i] * train[j]
        
        #频繁位置特征
        t = df.groupby('ship')['sign'].agg({'sign_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['sign_pca'].agg({'sign_pca_mode': lambda x: stats.mode(x)[0][0]}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        
        #角度统计特征
        t = group_feature(df, 'ship', 'd', ['mean'])
        train = pd.merge(train, t, on='ship', how='left')
        
        #针对不同的运动状态，进行速度的分箱统计
        def v_0(x):
            if len([i for i in x if i <= 2 and i > 0]) != 0:
                return [i for i in x if i <= 2 and i > 0]
            else:
                return [0]

        t = df.groupby('ship')['v'].agg({'v_count0': lambda x: len(v_0(x)) / len(x)}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['v'].agg({'v_mean0': lambda x: np.mean(v_0(x))}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')

        def v_1(x):
            if len([i for i in x if i > 6 and i <= 10]) != 0:
                return [i for i in x if i > 6 and i <= 10]
            else:
                return [0]

        t = df.groupby('ship')['v'].agg({'v_count1': lambda x: len(v_1(x)) / len(x)}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['v'].agg({'v_mean1': lambda x: np.mean(v_1(x))}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
    
        def v_2(x):
            if len([i for i in x if i <= 6 and i > 2]) != 0:
                return [i for i in x if i <= 6 and i > 2]
            else:
                return [0]

        t = df.groupby('ship')['v'].agg({'v_count2': lambda x: len(v_2(x)) / len(x)}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        t = df.groupby('ship')['v'].agg({'v_mean2': lambda x: np.mean(v_2(x))}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        
        #识别刺网的特征(不同的活动区域与总的记录区域的比值)
        t = df.groupby('ship')['sign'].agg({'sign_nunique': lambda x: len(set(x))/len(x)}).reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        def nunique_map(x):
            if x<0.2:
                y=1
            elif x>0.2 and x<0.63:
                y=2
            else:
                y=3
            return y
        train['sign_nunique']=train['sign_nunique'].map(nunique_map)
        
        #一些时间相关的特征
        df['hour_minutes'] = df['time'].dt.hour * 60 + df['time'].dt.minute
        t = df.groupby('ship')['hour_minutes'].agg('max').rename('hour_minutes_max').reset_index()
        train = pd.merge(train, t, on='ship', how='left')
        df['time_next']    = df.groupby('ship')['time'].shift(-1)
        df['time_last']    = df.groupby('ship')['time'].shift(1)
        df['time_to_next'] = df['time_next'] - df['time']
        df['time_to_next'] = df['time_to_next'].dt.total_seconds()
        #####################################################
        t = df.groupby('ship')['time_to_next'].agg('quantile').rename('time_to_next_quantile').reset_index()
        train = pd.merge(train,t,on = 'ship',how = 'left')
        #####################################################
        t = df.groupby('ship')['time_to_next'].agg('median').rename('time_to_next_median').reset_index()
        train = pd.merge(train,t,on = 'ship',how = 'left')
        df.drop(['hour_minutes','time_next', 'time_last', 'time_to_next'], axis=1, inplace=True)
        
        return train
    
    train_label = train_data.drop_duplicates('ship')
    
    del_list = ['x', 'y', 'v', 'd', '秒数', 'month', 'day', 'hour', 'weekday','x_pca', 'y_pca','sign','sign_pca']
    train_label = train_label.drop(del_list, axis=1)

    train_label = try_feature(train_data, train_label)

    train_df = train_label.drop(['label', 'ship'], axis=1)

    return train_df

In [ ]:
train_label = train_data.drop_duplicates('ship')
label=train_label['label']

In [ ]:
train_df0 = main(train_data)
train_df0.drop('time',axis = 1,inplace = True )

In [ ]:
#频繁位置活动船只的数量特征
train_df0['sign_mode_count']=train_df0['sign_mode'].map(train_df0['sign_mode'].value_counts())

#频繁位置 不同运动状态的速度统计特征
for i in ['v_mean0', 'v_mean1', 'v_mean2']:
    train_df0['sign_' + i + '_mean'] = train_df0.groupby('sign_mode')[i].transform('mean')
    
#保留两维有效数字
v_feature = [i for i in train_df0.columns if 'v' in i]
for i in v_feature:
    train_df0[i] = train_df0[i].map(lambda x: int((x + 0.005) * 100) / 100)

In [ ]:
#计算拐角特征
train_df0['guaijiao_mean']=[np.mean([j for j in i if j!=0]) for i in guai_list]
train_df0['guaijiao_mean1']=[np.mean([j for j in i if j<=0]) for i in guai_list]
train_df0['guaijiao_mean2']=[np.mean([j for j in i if j>=0]) for i in guai_list]
train_df0['guaijiao_count']=[len([j for j in i if j!=0])/len(i) for i in guai_list]
train_df0['guaijiao_count1']=[len([j for j in i if j<0])/len(i) for i in guai_list]
train_df0['guaijiao_count2']=[len([j for j in i if j>0])/len(i) for i in guai_list]
def guai_mode(x):
    mode_list=[]
    for i in x:
        if len([j for j in i if j!=0]) != 0:
            mode_list.append(stats.mode([j for j in i if j!=0])[0][0])
        else:
            mode_list.append(0)
    return mode_list
#train_df0['guaijiao_mode']=guai_mode(guai_list)

###### 模型选取了XGB模型，利用了贝叶斯优化进行调参

In [ ]:
def xgb_train(pout_train, target):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
    oof = np.zeros((len(pout_train), 3))
    feature_importance_df = pd.DataFrame()

    sum_num = 0
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(pout_train.values, target)):
        # trn_data,trn_label= pout_train.values[trn_idx], target[trn_idx]
        # val_data,val_label= pout_train.values[val_idx], target[val_idx]
        trn_data, trn_label = pout_train.iloc[trn_idx], target.iloc[trn_idx]
        val_data, val_label = pout_train.iloc[val_idx], target.iloc[val_idx]

        trn_data, val_data = trn_data.values, val_data.values
        trn_label, val_label = trn_label.values, val_label.values

        clf = XGBClassifier(objective='multi:softprob', max_depth=6, gamma=0, n_estimators=2000,
                                eta=0.02, min_child_weight=0.0007278877583954637, subsample=0.9683257011534013,
                                colsample_bytree=0.40030399408116674, reg_alpha=2.3027547889917508e-07,
                                reg_lambda=5.141887847497286e-05, num_class=3, random_state=2020)
        view_list = [(trn_data, trn_label), (val_data, val_label)]
        clf.fit(trn_data, trn_label, eval_set=view_list, eval_metric='mlogloss', early_stopping_rounds=300,verbose = False)
        # clf.fit(trn_data,trn_label,eval_set=view_list,eval_metric=myFeval,early_stopping_rounds=300)
        y_val, y_val_pred = val_label, clf.predict_proba(val_data)
        y_val_pvalue = np.argmax(y_val_pred, axis=1)
        f1_score_ = f1_score(y_val, y_val_pvalue, average='macro')
        print('第{}折的f1_score:'.format(fold_),f1_score_)
        print('-------------------------------------------------------------------------------------------------')
        oof[val_idx] = clf.predict_proba(pout_train.values[val_idx])

        sum_num += 1
        #predictions += clf.predict_proba(test_df.values)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = pout_train.columns
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions = predictions / sum_num

    oof_lgb_final = np.argmax(oof, axis=1)
    print("CV score: {}".format(f1_score(target.values, oof_lgb_final, average='macro')))
    print(confusion_matrix(target.values, oof_lgb_final))

    df_oof = pd.DataFrame()
    df_oof['label'] = oof_lgb_final
    print('实际的类别分布：',df_oof['label'].value_counts('normalize'))
    
    cols = (feature_importance_df[["Feature", "importance"]]
            .groupby("Feature")
            .mean()
            .sort_values(by="importance", ascending=False)[:150].index)
    best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('Features importance (averaged/folds)')
    plt.tight_layout()
    plt.show()

In [ ]:
xgb_train(train_df0, label)

###### 多个xgb种子投票，平衡方差

In [ ]:
def F1_score(y_true, y_pred):
    C = confusion_matrix(y_true, y_pred)

    TP_0 = C[0][0]
    FN_0 = C[0][1] + C[0][2]
    FP_0 = C[1][0] + C[2][0]
    TN_0 = sum(sum(C)) - TP_0 - FN_0 - FP_0
    precision_0 = TP_0 / (TP_0 + FP_0)
    recall_0 = TP_0 / (TP_0 + FN_0)
    F1_0 = 2 * precision_0 * recall_0 / (precision_0 + recall_0)

    TP_1 = C[1][1]
    FN_1 = C[1][0] + C[1][2]
    FP_1 = C[0][1] + C[2][1]
    TN_1 = sum(sum(C)) - TP_1 - FN_1 - FP_1
    precision_1 = TP_1 / (TP_1 + FP_1)
    recall_1 = TP_1 / (TP_1 + FN_1)
    F1_1 = 2 * precision_1 * recall_1 / (precision_1 + recall_1)

    TP_2 = C[2][2]
    FN_2 = C[2][0] + C[2][1]
    FP_2 = C[0][2] + C[1][2]
    TN_2 = sum(sum(C)) - TP_2 - FN_2 - FP_2
    precision_2 = TP_2 / (TP_2 + FP_2)
    recall_2 = TP_2 / (TP_2 + FN_2)
    F1_2 = 2 * precision_2 * recall_2 / (precision_2 + recall_2)
    return F1_0, F1_1, F1_2
def voting(test_sample):
    pred = []
    for i in range(test_sample.shape[1]):
        result = test_sample[:, i]
        value_counts = pd.Series(result).value_counts()
        pred_value = np.random.permutation(list(value_counts.index[value_counts == value_counts.iloc[0]]))[0]
        pred.append(pred_value)
    return np.array(pred)

def myFeval(preds, xgbtrain):
    label=xgbtrain.get_label()
    preds=np.argmax(preds, axis=1)
    f1=1-f1_score(label,preds,average='macro')
    return 'f1-score',f1

def xgb_train(pout_train, target, seed):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
    # folds = KFold(n_splits=5, shuffle=True, random_state=1234)
    oof = np.zeros((len(pout_train), 3))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(pout_train.values, target)):
        trn_data, trn_label = pout_train.iloc[trn_idx], target.iloc[trn_idx]
        val_data, val_label = pout_train.iloc[val_idx], target.iloc[val_idx]

        trn_data, val_data = trn_data.values, val_data.values
        trn_label, val_label = trn_label.values, val_label.values

        clf = XGBClassifier(objective='multi:softprob', max_depth=6, gamma=0, n_estimators=2000,
                                eta=0.02, min_child_weight=0.0007278877583954637, subsample=0.9683257011534013,
                                colsample_bytree=0.40030399408116674, reg_alpha=2.3027547889917508e-07,
                                reg_lambda=5.141887847497286e-05, num_class=3, random_state=seed)
        view_list = [(trn_data, trn_label), (val_data, val_label)]
        clf.fit(trn_data, trn_label, eval_set=view_list,eval_metric=myFeval, early_stopping_rounds=300, verbose=False)
        oof[val_idx] = clf.predict_proba(pout_train.values[val_idx])

    oof_lgb_final = np.argmax(oof, axis=1)

    f1, f2, f3 = F1_score(target.values, oof_lgb_final)
    print("CV score:", (f1 + f2 + f3) / 3)
        
    return oof, oof_lgb_final, (f1 + f2 + f3) / 3

def run_all_xgb(train_tdf, target):
    oof_class_list = []
    f1_list = []

    for kth, seed1 in enumerate([2020, 77, 66, 22, 33, 44, 55, 66, 88]):  # [32,42,47,1103,128,22,45][11,22,33,43,45,56,67]
        print(str(kth + 1), 'th seed begins----------------------')
        oof_num, oof_class, f1 = xgb_train(train_tdf, target, seed1)
        oof_class_list.append(oof_class)
        f1_list.append(f1)
    print('Mean f1 {}'.format(round(np.mean(f1_list), 5)))
    cv_f1 = metrics.f1_score(voting(np.vstack(tuple(oof_class_list))), target, average='macro')
    print('Voted CV f1{}'.format(round(cv_f1, 5)))

print('*' * 5 + '正在训练模型')

run_all_xgb(train_df0, label)